In [0]:
import sys
import json

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import explode
from pyspark.sql.functions import *
from pyspark.sql.types import StringType



# constants
inputFilePath = "abfss://{}@{}.dfs.core.windows.net/{}/{}/????????\-*"
storageAccount = "{}.dfs.core.windows.net"
outputFilePath = "https://{}.dfs.core.windows.net/{}/{}"
csvOutputPath = "abfss://{}@{}.dfs.core.windows.net/{}/"
metadataFilePath = "abfss://{}@{}.dfs.core.windows.net/{}/model.json"
manifestPath = "{}/{}/default.manifest.cdm.json"
dataSourceColumnName = "sourceOfOrigin"
dataSourceColumnValue = "Office365"

def writeDataframeToCdmFormat(dataframe, manifestPath, storage, cdmModelName, cdmEntityName, appId, appKey, tenantId):
    """
    This method write dataframe in CDM format to the outputPath

    :param dataframe: dataframe to write
    :type dataframe: object
    :param outputPath: output path to write data
    :type outputPath: string
    :param cdmModelName: model name in CDM metadata
    :type cdmModelName: string
    :param cdmEntityName: entity name in CDM metadata
    :type cdmEntityName: string
    :param appID: AAD application Id to access storage account
    :type appID: string
    :param appKey: AAD application key to access storage account
    :type appKey: string
    :param tenantId: AAD application tenant Id
    :type tenantId: string
    """
    dataframe.repartition(1).write.format("com.microsoft.cdm") \
			.option("storage", storage)\
			.option("manifestPath",manifestPath)\
      .option("entity", cdmEntityName) \
      .option("appId", appId) \
      .option("appKey", appKey) \
      .option("tenantId", tenantId) \
      .mode("overwrite") \
			.save()



def createAppointmentDataframe(eventDf):
    """
    This method create Appointment dataframe for CDM

    :param eventDf: event dataframe
    :type eventDf: object
    """
    appointmentColumns = [
      "body.content as body","createdDateTime", "end.datetime as end","hasAttachments",
      "iCalUId","id as appointmentId","importance","isAllDay","isCancelled","isReminderOn","lastModifiedDateTime",
      "onlineMeetingUrl","originalEndTimeZone","originalStart","originalStartTimeZone","reminderMinutesBeforeStart",
      "responseRequested","sensitivity","seriesMasterId","showAs", "start.datetime as start","subject","type"
      ]

    return eventDf.selectExpr(appointmentColumns).withColumn(dataSourceColumnName, lit(dataSourceColumnValue))

def createEmailDataframe(messageDf):
    """
    This method create Email dataframe for CDM

    :param messageDf: message dataframe
    :type messageDf: object
    """
    emailColumns = [
      "uniqueBody.content as body","conversationId","createdDateTime","hasAttachments","id as emailId","importance","inferenceClassification",
      "internetMessageId","isDeliveryReceiptRequested","isDraft","isRead","isReadReceiptRequested","lastModifiedDateTime",
      "parentFolderId","receivedDateTime","sentDateTime","subject"
      ]

    return messageDf.selectExpr(emailColumns).withColumn(dataSourceColumnName, lit(dataSourceColumnValue))

def createPersonDataframe(managerDf, userDf):
    """
    This method create Person dataframe for CDM

    :param managerDf: manager dataframe
    :type managerDf: object
    :param userDf: user dataframe
    :type userDf: object
    """
    managerColumns = [
      "id as managerId", "puser"
    ]
    userColumns = [
      "birthday","companyName","createdDateTime","department","displayName",
      "givenName","hireDate","id as personId","jobTitle","mail","officeLocation","preferredLanguage","preferredName","surname","usageLocation","userPrincipalName","userType"
    ]
	
    managerDf = managerDf.selectExpr(managerColumns)
    userDf = userDf.selectExpr(userColumns)
    
    # Append manager id to user data
    return userDf.join(managerDf, userDf.personId == managerDf.puser, how='left').drop("puser")

def createAppointmentParticipantsDataframe(eventDf):
    """
    This method create AppointmentParticipant dataframe for CDM

    :param eventDf: event dataframe
    :type eventDf: object
    """
	
    # Get appointment attendees
    attendeesDf = eventDf.select(eventDf.id.alias("appointmentId"),explode("attendees").alias("attendees"))
    if(not attendeesDf.rdd.isEmpty()):
      attendeesDf = attendeesDf.select("appointmentId","attendees.emailAddress.address","attendees.status.response").withColumn("isOrganizer", lit(False))

    # Get appointment organizer   
    organizerDf = eventDf.select(eventDf.id.alias("appointmentId"), "organizer.emailAddress.address").withColumn("response", lit(None).cast(StringType())).withColumn("isOrganizer", lit(True))

    # Merge the attendees and organizer
    participantsDf = organizerDf
    if(not attendeesDf.rdd.isEmpty()):
	    participantsDf = participantsDf.unionByName(attendeesDf)
    
    return participantsDf

def createEmailParticipantsDataframe(messageDf):
    """
    This method create EmailParticipant dataframe for CDM

    :param messageDf: message dataframe
    :type messageDf: object
    """
	
    # Get to recipients in email
    toRecipientsDf = messageDf.select(messageDf.id.alias("emailId"),explode("toRecipients").alias("toRecipients"))
    if(not toRecipientsDf.rdd.isEmpty()):
      toRecipientsDf = toRecipientsDf.select("emailId","toRecipients.emailAddress.address").withColumn("isSender", lit(False)).withColumn("recipientType", lit("To"))

    # Get cc recipients in email
    ccRecipientsDf = messageDf.select(messageDf.id.alias("emailId"),explode("ccRecipients").alias("ccRecipients"))
    if(not ccRecipientsDf.rdd.isEmpty()):
      ccRecipientsDf = ccRecipientsDf.select("emailId","ccRecipients.emailAddress.address").withColumn("isSender", lit(False)).withColumn("recipientType", lit("Cc"))

    # Get bcc recipients in email
    bccRecipientsDf = messageDf.select(messageDf.id.alias("emailId"),explode("bccRecipients").alias("bccRecipients"))
    if(not bccRecipientsDf.rdd.isEmpty()):
      bccRecipientsDf = bccRecipientsDf.select("emailId","bccRecipients.emailAddress.address").withColumn("isSender", lit(False)).withColumn("recipientType", lit("Bcc"))

    # Get sender in email  
    senderDf = messageDf.select(messageDf.id.alias("emailId"),"sender.emailAddress.address").withColumn("isSender", lit(True)).withColumn("recipientType", lit(None).cast(StringType()))

    # Merge to, cc, bcc and sender
    participantsDf = senderDf
    if(not toRecipientsDf.rdd.isEmpty()):
	    participantsDf = participantsDf.unionByName(toRecipientsDf)
    if(not ccRecipientsDf.rdd.isEmpty()):
	    participantsDf = participantsDf.unionByName(ccRecipientsDf)
    if(not bccRecipientsDf.rdd.isEmpty()):
	    participantsDf =participantsDf.unionByName(bccRecipientsDf)
      
    return participantsDf
	
def createEmailAddressDataframe(personDf):
    """
    This method create EmailAddress dataframe for CDM

    :param personDf: person dataframe
    :type personDf: object
    """
    mailDf = personDf.select("personId", personDf.mail.alias("emailAddress")).where(personDf.mail.isNotNull())
    upnDf = personDf.select("personId", personDf.userPrincipalName.alias("emailAddress")).where(personDf.userPrincipalName.isNotNull())
    
    return mailDf.unionByName(upnDf).distinct()

def createPhoneNumberDataframe(userDf):
    """
    This method create PhoneNumber dataframe for CDM

    :param userDf: person dataframe
    :type userDf: object
    """
    phoneNumberType = "phoneNumberType"
    businessPhoneDf = userDf.selectExpr("id as personId", "businessPhones[0] as phoneNumber").where(userDf.businessPhones[0].isNotNull()).withColumn(phoneNumberType, lit("Business"))
    mobilePhoneDf = userDf.selectExpr("id as personId", "mobilePhone as phoneNumber").where(userDf.mobilePhone.isNotNull()).withColumn(phoneNumberType, lit("Mobile"))
    
    return businessPhoneDf.unionByName(mobilePhoneDf)
	
def createPhysicalAddressDataframe(userDf):
    """
    This method create PhoneNumber dataframe for CDM

    :param userDf: person dataframe
    :type userDf: object
    """
    addressColumns = [
      "id as personId","city","country","officeLocation","postalCode","state","streetAddress"
	  ]
    return userDf.selectExpr(addressColumns).where(userDf.country.isNotNull())

def appendExternalUsers(personDf, appointmentParticipantsDf, emailParticipantsDf, sqlContext):
    """
    This method create MailParticipants dataframe for CDM

    :param personDf: Person dataframe
    :type personDf: object
    :param appointmentParticipantsDf: AppointmentParticipants dataframe
    :type appointmentParticipantsDf: object
    :param emailParticipantsDf: MailParticipants dataframe
    :type emailParticipantsDf: object
    :param sqlContext: SQL Context
    :type sqlContext: object
    """
    emptyDf = sqlContext.createDataFrame([], personDf.schema).drop("personId")

    # Get all email address from appointments and emails
    externalUserDf = appointmentParticipantsDf.select(appointmentParticipantsDf.address.alias("mail")).distinct().unionByName(emailParticipantsDf.select(emailParticipantsDf.address.alias("mail")).distinct())

    # Remove known mail and userPrincipalName in AAD user data
    externalUserDf = externalUserDf.subtract(personDf.select("mail"))
    externalUserDf = externalUserDf.subtract(personDf.selectExpr("userPrincipalName as mail"))

    # Construct new pserson data for unknown email address
    externalUserDf = externalUserDf.join(emptyDf, "mail", how='left')
    externalUserDf = externalUserDf.withColumn("personId", externalUserDf.mail)
  
    return externalUserDf.unionByName(personDf)

def updatePersonId(emailAddressDf, participantsDf):
    """
    This method update participants dataframe with PersonId for CDM

    :param emailAddressDf: emailAddressDf dataframe
    :type emailAddressDf: object
    :param participantsDf: Participants dataframe
    :type participantsDf: object
    """
    return participantsDf.join(emailAddressDf.select("emailAddress","personId"), participantsDf.address == emailAddressDf.emailAddress, how='left').drop("address","emailAddress")

if __name__ == "__main__":

  # create context with Spark configuration
  conf = SparkConf().setAppName("JsonToCdm")
#   sc = SparkContext(conf=conf)
  sc = SparkContext.getOrCreate(conf=conf)
  sqlContext = SQLContext(sc)
  
  # read input parameters from Widgets

  dbutils.widgets.text("OfficeDataFileSystem", "", "")
  dbutils.widgets.text("StorageAccountName", "","")
  extrationFS = dbutils.widgets.get("OfficeDataFileSystem")
  storageAccountName = dbutils.widgets.get("StorageAccountName")

  dbutils.widgets.text("AppId", "", "")
  dbutils.widgets.text("AppKey", "","")
  dbutils.widgets.text("TenantId", "","")
  appId = dbutils.widgets.get("AppId")
  appKey = dbutils.widgets.get("AppKey")
  tenantId = dbutils.widgets.get("TenantId")

  dbutils.widgets.text("CdmDataFileSystem", "", "")
  dbutils.widgets.text("CdmModelName", "","")
  cdmDataFS = dbutils.widgets.get("CdmDataFileSystem")
  cdmModelName = dbutils.widgets.get("CdmModelName")

  dbutils.widgets.text("RunId", "","")
  pipelineId = dbutils.widgets.get("RunId")

  dbutils.widgets.text("EventDatasetFolder", "", "")
  dbutils.widgets.text("ManagerDatasetFolder", "","")
  dbutils.widgets.text("MessageDatasetFolder", "","")
  dbutils.widgets.text("UserDatasetFolder", "","")
  eventDataset = dbutils.widgets.get("EventDatasetFolder")
  managerDataset = dbutils.widgets.get("ManagerDatasetFolder")
  messageDataset = dbutils.widgets.get("MessageDatasetFolder")
  userDataset = dbutils.widgets.get("UserDatasetFolder")

  
# # #Initializing for test purposes

# Define the variables used for creating connection strings
# initialize the blank variables with correct values
# Application (Client) ID
  appId = ""
# Application (Client) Secret Key
  appKey = ""
# Directory (Tenant) ID
  tenantId = ""
  storageAccountName = ""
  runId = ""
  pipelineId = runId

  userDataset = "user"
  managerDataset = "manager"
  messageDataset = "message"
  eventDataset = "event"

  extrationFS = "json"
  cdmDataFS = "cdm"
  csvDataFS = "csv"

# Storage account authentication

  endpoint = "https://login.microsoftonline.com/" + tenantId + "/oauth2/token"

# Connecting using Service Principal secrets and OAuth
  configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": appId,
           "fs.azure.account.oauth2.client.secret": appKey,
           "fs.azure.account.oauth2.client.endpoint": endpoint}

  [spark.conf.set(key, value) for (key, value) in configs.items()]


   # read the json file
  eventDf = spark.read.json(inputFilePath.format(extrationFS, storageAccountName, pipelineId, eventDataset))
  managerDf = spark.read.json(inputFilePath.format(extrationFS, storageAccountName, pipelineId, managerDataset))
  messageDf = spark.read.json(inputFilePath.format(extrationFS, storageAccountName, pipelineId, messageDataset))
  userDf = spark.read.json(inputFilePath.format(extrationFS, storageAccountName, pipelineId, userDataset))

  # schmea transformation for appointment
  appointmentDf = createAppointmentDataframe(eventDf)
  appointmentDf.show()
  
  # schmea transformation for email
  emailDf = createEmailDataframe(messageDf)
  emailDf.show()

  # schmea transformation for phoneNumber
  phoneNumberDf = createPhoneNumberDataframe(userDf)
  phoneNumberDf.show()
  
  # schmea transformation for physicalAddress
  physicalAddressDf = createPhysicalAddressDataframe(userDf)
  physicalAddressDf.show()

  # schmea transformation for person
  personDf = createPersonDataframe(managerDf, userDf)
  appointmentParticipantDf = createAppointmentParticipantsDataframe(eventDf)
  emailParticipantDf = createEmailParticipantsDataframe(messageDf)
  personDf = appendExternalUsers(personDf, appointmentParticipantDf, emailParticipantDf, sqlContext)
  personDf.show()
  
  # schmea transformation for emailAddress
  emailAddressDf = createEmailAddressDataframe(personDf)
  emailAddressDf.show()

  # schmea transformation for appointmentParticipant
 
  appointmentParticipantDf = updatePersonId(emailAddressDf, appointmentParticipantDf)
  appointmentParticipantDf.show()

  # schmea transformation for emailParticipant
  emailParticipantDf = updatePersonId(emailAddressDf, emailParticipantDf)
  emailParticipantDf.show()
  
  # Write transformed data into CDM format

  outputLocation = manifestPath.format(cdmDataFS, pipelineId)

  storage = storageAccount.format(storageAccountName)

  # CDM format Export  
  writeDataframeToCdmFormat(appointmentDf, outputLocation, storage, cdmModelName, "Appointment", appId, appKey, tenantId)
  writeDataframeToCdmFormat(appointmentParticipantDf, outputLocation, storage, cdmModelName, "AppointmentParticipant", appId, appKey, tenantId)
  writeDataframeToCdmFormat(personDf, outputLocation, storage, cdmModelName, "Person", appId, appKey, tenantId)
  writeDataframeToCdmFormat(emailParticipantDf, outputLocation, storage, cdmModelName, "EmailParticipant", appId, appKey, tenantId)
  writeDataframeToCdmFormat(emailDf, outputLocation, storage, cdmModelName, "Email", appId, appKey, tenantId)
  writeDataframeToCdmFormat(phoneNumberDf, outputLocation,storage, cdmModelName, "PhoneNumber", appId, appKey, tenantId)
  writeDataframeToCdmFormat(physicalAddressDf, outputLocation,storage, cdmModelName, "PhysicalAddress", appId, appKey, tenantId)
  writeDataframeToCdmFormat(emailAddressDf, outputLocation,storage, cdmModelName, "EmailAddress", appId, appKey, tenantId)

In [0]:
# #declare data lake path where we want to write the data


outputLocation = csvOutputPath.format(csvDataFS, storageAccountName, pipelineId)

# #write as csv data
appointmentDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"appointment")
appointmentParticipantDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"AppointmentParticipant")
personDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"person")
emailParticipantDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"emailParticipant")
emailDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"email")
phoneNumberDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"phoneNumber")
physicalAddressDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"physicalAddress")
emailAddressDf.write.mode("overwrite").option("header",True).option("delimiter",",").csv(outputLocation+"emailAddress")

